In [7]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
import cv2
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import train_test_split
import os
pre_processed_img_dir = 'gear_images_converted'
pre_processed_label_dir = 'gear_labels_converted'

label_dict={'axes':0,'boots':1,'carabiners':2,'crampons':3,'gloves':4,'hardshell_jackets':5,'harnesses':6,'helmets':7
           ,'insulated_jackets':8,'pulleys':9,'rope':10,'tents':11}




In [17]:
added_img_array= []
added_label_array= []
for subdir, dirs, files in os.walk(pre_processed_img_dir):

    for dir in dirs:

        for file in os.listdir(pre_processed_img_dir + '/' + dir):

            img_array = cv2.imread(pre_processed_img_dir + '/' + dir + '/' + file)

            label_array=label_dict[dir]
            added_img_array.append(img_array)
            added_label_array.append(label_array)


added_label_array_array=np.delete(added_label_array,0)
added_label_array = np.asarray(added_label_array)
added_img_array = np.concatenate([arr[np.newaxis] for arr in added_img_array])
#added_img_array = np.reshape(added_img_array, (added_img_array.shape[0], -1))

print(added_img_array.shape)
print(added_label_array.shape)

added_img_array=added_img_array.astype(float)
added_label_array=added_label_array.astype(float)


(2122, 128, 128, 3)
(2122,)


In [22]:
added_label_array=np.transpose(added_label_array)
x_train, x_test, y_train, y_test = train_test_split(added_img_array, added_label_array, test_size=0.33, random_state=42)

In [29]:

batch_size = 128
num_classes = 12
epochs = 12


# input image dimensions
img_rows, img_cols = 128, 128

'''
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
'''
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
input_shape = (img_rows, img_cols, 3)



model = Sequential()
'''
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
'''
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: Error when checking target: expected dense_22 to have 4 dimensions, but got array with shape (1421, 12, 12, 12, 12, 12)